# 大语言模型量化处理
测试模型直接为：Qwen2-0.5B-Instruct

## 使用llama.cpp量化  
> https://qwen.readthedocs.io/zh-cn/latest/quantization/llama.cpp.html

比如说对于`Qwen2.5-1.5B-Instruct`模型，一般使用是：
```python
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))
```
为了实现GGUF格式转换，按照下面步骤进行（为了步骤简洁，内容全部放到终结里面）
### 1、无校准量化GGUF
```python
# 首先创建GGUF文件  转换文件
python convert-hf-to-gguf.py 模型名称/地址 --outtype bf16 --outfile Qwen3-1.5B-BF16.gguf 
# 其中的 outtype后面的数据类型可以是：bf16 f32等
```
在执行上面代码之后直接将模型量化到`8 bit`：
```python
./build/bin/llama-quantize --quantize --model Qwen3-1.5B-F16.gguf --output Qwen3-1.5B-Q8_0.gguf --type Q8_0
```
`Q8_0`是一个量化预设的代号
### 2、使用重要性矩阵量化GGUF
```python
# 首先
./llama-imatrix -m Qwen3-1.5B-F16.gguf -f calibration-text.txt --chunk 512 -o Qwen3-1.5B-imatrix.dat -ngl 80
# 
./llama-quantize --imatrix Qwen3-1.5B-imatrix.dat Qwen3-1.5B-F16.gguf Qwen3-1.5B-Q4_K_M.gguf Q4_K_M
```
### 两种量化方式总结
* **首先解释代码使用**：

在此之前可以使用`python convert_hf_to_gguf.py --help`了解一些这个脚本都有哪些参数，`python convert_hf_to_gguf.py --print-supported-models`直接看一下支持哪些模型进行量化，一般而言模型名称就和`config.json`中是一致的比如`Qwen2ForCausalLM`。量化开始：**首先**，在下载得到模型权重之后，在`cache_dir`里面就会有模型权重，比如说上面过程中模型权重就会保存为：`/Model/models--Qwen--Qwen2.5-1.5B-Instruct`，那么在创建以及转换文件过程中就需要将其中的 **模型名称/地址**进行替换，那么**完整带脚本**就是：`python convert_hf_to_gguf.py /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct --outfile /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf --outtype bf16 --verbose`
**不过**，上面代码还是会出错，因为实际模型文件在`code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306`因此需要将上面的文件路径改为：`/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/Model/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306`最后输出：
```bash
...
INFO:hf-to-gguf:Model successfully exported to /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf
```
就表示成功了，而后可以通过`./build/bin/llama-quantize --help`去检查支持哪些参数，**并且进行量化**：`./build/bin/llama-quantize /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-BF16.gguf /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf q8_0`。不过还有一点有意思的，比如说我的模型一般都会用lora进行微调，我需要对lora微调模型进行处理，类似的：
```bash
python convert_lora_to_gguf.py /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/test-lora/ --outfile /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf --outtype bf16 --verbose

./build/bin/llama-quantize /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf /root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-Q8_0.gguf q8_0
```
### 使用GGUF量化模型
```bash
pip install llama-cpp-python
# 直接去 https://github.com/abetlen/llama-cpp-python/releases 下载对应的版本 和flash-attn一样
pip install llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl
```
这样一来就安装好了就可以直接加载GGUF模型权重了（和非量化模型推理10.8s，**量化后推理：3.5s**）：
```python
from llama_cpp import Llama

llm = Llama(
    model_path="/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf",
    n_ctx=8192,
    n_gpu_layers=-1,
    chat_format="chatml",
    verbose=False
)

messages = [
    {"role": "system", "content": "你是一个有帮助的助手，用简洁的中文回答。"},
    {"role": "user",   "content": "晚上睡不着怎么办"}
]

# 生成
response = llm.create_chat_completion(
    messages,
    max_tokens=300,
    temperature=0.75,
    top_p=0.95,
    repeat_penalty=1.1,
    stream=False  # 改成 True 即可流式输出
)

print("AI 回答：")
print(response["choices"][0]["message"]["content"])
```

In [8]:
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

def load_model():
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                                cache_dir= '/root/autodl-tmp/Model/', 
                                                mirror='https://hf-mirror.com')
    return model, tokenizer

def load_GGUF(lora_path=None):
    from llama_cpp import Llama

    llm = Llama(
        model_path="/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Qwen3-1.5B-Q8_0.gguf",
        lora_path= lora_path,
        n_ctx=8192,
        n_gpu_layers=-1,
        chat_format="chatml",
        verbose=False
    )
    return llm
messages = [
    {"role": "system", "content": "你是一个有帮助的助手，用简洁的中文回答。"},
    {"role": "user",   "content": "晚上睡不着怎么办?"}
]
model, tokenizer = load_model()
model_gguf = load_GGUF()
model_gguf_lora = load_GGUF(lora_path= '/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/GGUF1-Lora-Qwen3-1.5B-BF16.gguf')

llama_context: n_ctx_per_seq (8192) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (8192) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [ ]:
# 生成测试lora
import os
import torch
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM

rank = 8  
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"] 
output_dir = "/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/test-lora/"  
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct",
    torch_dtype=torch.float32,
    device_map="cpu",
    cache_dir= '/root/autodl-tmp/Model/', 
    low_cpu_mem_usage=True
)

lora_config = LoraConfig(
    r=rank,
    lora_alpha=16,
    target_modules=target_modules,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
    init_lora_weights=False
)

peft_model = get_peft_model(model, lora_config)
peft_model.save_pretrained(output_dir, save_embedding_layers=False)

`torch_dtype` is deprecated! Use `dtype` instead!


全零 LoRA adapter 已生成到：/root/autodl-tmp/Code/Big-Yellow-J.github.io/code/Python/DFModelCode/DF_acceralate/tmp/test-lora/
文件列表：
 - README.md
 - adapter_model.safetensors
 - adapter_config.json

所有 LoRA 权重均为 0，加载后效果等同于基模型（可用于测试 GGUF 转换/量化/加载链路）
警告：base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight 不为零！
警告：base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight 不为零！
警告：base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight 不为零！
警告：base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight 不为零！
警告：base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight 不为零！
警告：base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight 不为零！
警告：base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight 不为零！
警告：base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight 不为零！
警告：base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight 不为零！
警告：base_model.model.model.layers.0.mlp.gate_proj.lo

In [9]:
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

可以试试深呼吸、听轻音乐、喝杯热牛奶或泡个热水澡等方法。如果经常失眠，建议咨询医生。<|im_end|>


In [10]:
response = model_gguf.create_chat_completion(
    messages,
    max_tokens=300,
    temperature=0.75,
    top_p=0.95,
    repeat_penalty=1.1,
    stream=False  # 改成 True 即可流式输出
)

print("AI 回答：")
print(response["choices"][0]["message"]["content"])

AI 回答：
可尝试用热水泡脚，听听轻音乐，或者看看电视。
